# Amplifier classes
* [Linear](https://eng.libretexts.org/Bookshelves/Electrical_Engineering/Electronics/Microwave_and_RF_Design_V%3A_Amplifiers_and_Oscillators_(Steer)/02%3A_Linear_Amplifiers/2.05%3A_Class_A_AB_B_and_C_Amplifiers?readerView) : A, B, C
* [Switched mode](https://eng.libretexts.org/Bookshelves/Electrical_Engineering/Electronics/Microwave_and_RF_Design_V%3A_Amplifiers_and_Oscillators_(Steer)/04%3A_Power_Amplifiers/4.03%3A_Switching_Amplifiers_Classes_D_E_and_F?readerView) : D, E, F
  * Class D : output capacitance of transistor is a loss element
  * **Class E** : output capacitance of transistor used in a tuned circuit, filter odd harmonics away (=loss)
  * Class F : improves on class E by adding tuned circuits for the odd harmonics as well.  In practice for QRP-use, the benefit is limited.

# Class-E topologies
[Scott Howard](https://raw.githubusercontent.com/KD9PDP/Minimalist-Class-E/master/PDF/minimalist_class_e.pdf) has compared Class E-amplifiers.  To have enough rejection of the harmonics, a Pi-match or T-match circuit is needed.  The Pi-match circuit is preferred because it gives more harmonic rejection and uses smaller inductor values.
We'll try both topologies to see what's best in our case.

# KD9PDP : Class E-amplifier with Pi-match notch filter
This topology is also used in the [(tr)uSDX receiver](./doc/truSDX_Main-RF_Boards_with_Notes_v1.0n.pdf).

![NM0S topology, image from KD9PDP paper](./doc/NM0S_pimatchnotch.png)

In [1]:
# Calculations for Pi-match with notch-filter
# according to KD9PDP

import math

# Design parameters
VCC = 12.0  # V
Pout = 5   # W
f = 27e6   # Hz
R1 = 50    # Ohm

Rl = 0.58 * math.pow(VCC, 2) / Pout  # Ohm, load seen by the MOSFET drain
Q = 3 + math.sqrt(Rl/5 - 1)

print("Q = %.2f" % Q)
print("Rl = %.2f Ohm" % Rl)

ω = 2*math.pi*f  # rad/s
L1 = 2.84 * Rl / ω          # H, minimum value for L1
C1 = 1 / (5.4466 * ω * Rl)  # F
Cf = 1 / (Q * ω * Rl)      # F
Ltot = (Q + 1.1525) * Rl / ω  # H
C2 = (Q-3) / (ω * Rl)         # F
L2 = 15 * Q / (4 * ω)       # H
C4 = 1 / (15*Q*ω)        # F
C3 = 3 / (50*ω)          # F

# LTSpice results
I_R1 = 0.349  # A
I_L1 = 0.531  # A
η = math.pow(I_R1,2)* R1 / (VCC*I_L1)
print("η = %.2f" % η)

print("L1 = %.2f uH" % (L1*1e6))
print("C1 = %.2f pF" % (C1*1e12))
print("Cf = %.2f pF" % (Cf*1e12))
print("Ltot = %.2f uH" % (Ltot*1e6))
print("C2 = %.2f pF" % (C2*1e12))
print("L2 = %.2f uH" % (L2*1e6))
print("C4 = %.2f pF" % (C4*1e12))
print("C3 = %.2f pF" % (C3*1e12))


Q = 4.53
Rl = 16.70 Ohm
η = 0.96
L1 = 0.28 uH
C1 = 64.79 pF
Cf = 77.90 pF
Ltot = 0.56 uH
C2 = 539.91 pF
L2 = 0.10 uH
C4 = 86.75 pF
C3 = 353.68 pF


## Results
* I(L1) = 531mArms
* I(R1) = 349mArms
* I(Ltot) = 608mArms
* I(L2) = 1.48Arms
* η = 96%
* Harmonics:
  * 26.866MHz : 21.72dB
  * 53.766MHz : -32.05dB
  * 80.633MHz : -25.28dB
  * 134.4MHz : -27.36dB

# Conclusion
Very high inductor currents will cause high I²R-losses.  Filter performance is inferior to WA0ITP design.

# WA0ITP : Class-E amplifier with Pi-match notch filter
Calculation based on [WA0ITP's spreadsheet of the NM0S equations](http://www.wa0itp.com/classeampdesignrevb.xls)
![wa0itp pi-match notch design](./doc/WA0ITP_pimatchnotch.png)

The RdsOn and Coss have been set to zero to simulate a perfect switch.

## Results
* I(L1) = 431mArms
* I(R1) = 315mArms
* I(L2) = 791mArms
* I(L3) = 1.07Arms
* η = 96%
* Harmonics:
  * 26.87MHz : 20.86dB
  * 53.78MHz : -40.67dB
  * 80.63MHz : -49.90dB
  * 134.4MHz : -23.47dB

In [2]:
# LTSpice results
I_R1 = 0.315  # A
I_L1 = 0.431  # A
η = math.pow(I_R1,2)* R1 / (VCC*I_L1)
print("η = %.2f" % η)

η = 0.96


# Conclusion
Preferable to KD9PDP Pi-match design because of better filtering and lower inductor current.

# T-match notch filter
![t-match notch design](./doc/tmatchnotch.png)

In [3]:
# Calculations for T-match with notch-filter according to KD9PDP
# Design parameters
VCC = 12.0  # V
Pout = 5   # W
f = 27e6   # Hz
Rant = 50    # Ohm
Q = 5

# Ltot and Cf are equal to the ones calculated for the Pi-match

Rl = 0.58 * math.pow(VCC, 2) / Pout  # Ohm, load seen by the MOSFET drain
Rv = Rl * (math.pow(Q, 2) + 1) # virtual load of the T-network
Q1 = math.sqrt(Rv/Rl - 1) # Q-factor of the T-network, seen from the MOSFET drain
Q2 = math.sqrt(Rv/Rant - 1) # Q-factor of the T-network, seen from the antenna

ω = 2*math.pi*f  # rad/s
L2 = Q1 * Rl / ω  # H
L3 = Q2 * Rant / ω  * 3/4 # H
C2 = 1 / (Rv * ω) * (Q1+Q2) # F
C3 = 1 / (math.pow(2*ω,2) * L3) # F

print('L2 = {:.2f}µH'.format(L2*1e6))
print('L3 = {:.2f}µH'.format(L3*1e6))
print('C2 = {:.2f}pF'.format(C2*1e12))
print('C3 = {:.2f}pF'.format(C3*1e12))

# LTSpice results
I_R1 = 0.328  # A
I_L1 = 0.473  # A
η = math.pow(I_R1,2)* R1 / (VCC*I_L1)
print("η = %.2f" % η)


L2 = 0.49µH
L3 = 0.61µH
C2 = 105.49pF
C3 = 14.17pF
η = 0.95


## Results
![waveforms from voltage over switch and current through switch](./doc/tmatchnotch_waveforms.png)

* I(L1) = 473mArms
* I(R1) = 328mArms
* I(Ltot) = 560mArms
* I(L3) = 435mArms
* η = 95%
* Harmonics:
  * 26.87MHz : 21.19dB
  * 53.77MHz : -41.19dB
  * 80.64MHz : -22.7dB
  * 134.4MHz : -29.43dB

## Conclusion
3rd harmonic suppression isn't great, but for ETSI we need an extra filter anyway.
This design has the advantage that it has the lowest inductor currents and the least amount of components.

# References
* [Search for the Minimalist QRP Class E Amplifier Filter Typology, KD9PDP](https://raw.githubusercontent.com/KD9PDP/Minimalist-Class-E/master/PDF/minimalist_class_e.pdf)
  * Must read
  * Good explanation of different Class-E topologies with calculations
* [Class E Amplifier Design Spreadsheet](http://wa0itp.com/class%20e%20design.html)
  * Implementation of David Cripe (NM0S) equations.
* [Notes on designing Class-E RF power ampliﬁers, George/Bill Slade](https://www.researchgate.net/publication/320623200)
  * Implementation of what KD9PDP describes as "2.1.1 “Power Boost” Class E".
  * Presented design requires high Q BPF (not described) to reduce harmonics enough.
* [Class E Amplifiers, Dan Tayloe, N7VE](http://norcalqrp.org/files/Class_E_Amplifiers.pdf)
  * protection circuits for open and shorted loads

## Other references (less interesting)
* [Modeling Class-E RF amplifiers](https://k6jca.blogspot.com/2011/01/modeling-class-ef-rf-power-amplifiers.html)
* [An Introduction to the uSDX transceiver kit](https://qrper.com/2020/09/an-introduction-to-the-usdx/)